In [1]:
import numpy as np
from sklearn.preprocessing import normalize
from itertools import combinations
from math import copysign

In [2]:
def create_random(count, dimension):
    vectors = np.zeros((count,dimension))
    for i in range(count):
        v = (2*np.random.rand(dimension))-1
        vectors[i] = v / (v**2).sum()**0.5
    return vectors

In [3]:
def find_projection(x, signature_size, bucket_count):
    # Normalize points
    x = normalize(x, axis=0, norm='l1')
    # Create hash tables
    hash_tables = []
    for i in range(signature_size):
        hash_tables.append({})
    
    # Create a random line in space
    hash_lines = create_random(count=signature_size, dimension=np.shape(x)[1])
    sign = lambda x : int(copysign(1, x))
    
    # Apply each hash function
    for j in range(signature_size):
            # To each data point
            for m in range(np.shape(x)[0]):
                
                # Project point onto random line
                projection = np.dot(x[m], hash_lines[j]) * hash_lines[j]
                
                # Find its bucket according to bucket_count, we have 1/bucket_count intervals on line
                hash_val = int((sign(projection[0]) * np.linalg.norm(projection)+1)*(bucket_count/2))

                # Put the index of item to the hash table j with generated hash value
                if hash_val not in hash_tables[j]:
                    hash_tables[j][hash_val] = [m]
                else:
                    hash_tables[j][hash_val].append(m)
    return hash_tables

In [4]:
def generate_candidates(hash_tables):
    candidate_pairs = []
    for table in hash_tables:
        for row in table:
            bucket = table[row]
            pairs = combinations(bucket, 2)
            for elements in pairs:
                candidate_pairs.append(tuple(sorted(elements)))
    
    # Take most occuring pairs, not all of them
    result = []
    for e in candidate_pairs:
        count = candidate_pairs.count(e)
        if count > len(hash_tables)/20:
            result.append(e)
    return np.array(set(result))

In [14]:
c = np.array([[1,5,70],[20,3,7],[3000,1100,-6000], [3,11,12]])
x = find_projection(c, signature_size=60, bucket_count=1000)
y = generate_candidates(x)
y

array({(0, 1), (0, 3), (1, 3)}, dtype=object)